In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    probs = np.array(items_weights['weight'])
    probs /= probs.sum()
    recs = np.random.choice(items, size=n, replace=False, p=probs)
    
    return recs.tolist()

In [5]:
%%time
pd.set_option('use_inf_as_na', True)

# your_code
items = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items['log_item_sales'] = np.log(items['sales_value']).fillna(0)
items['weight'] = items['log_item_sales'] / items['log_item_sales'].sum()
items = items.drop(columns=['sales_value', 'log_item_sales'])
items.loc[items.weight < 0, 'weight'] = 0

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, n=5))
result.head()

C:\Users\horne\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Wall time: 5.37 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9935616, 6514144, 1256920, 15596908, 12648296]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13211924, 1027735, 9837432, 921489, 1119243]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[9297064, 883562, 14025680, 897731, 1044434]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[12456946, 10121739, 967957, 12524439, 851067]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[8293355, 13158602, 1070983, 1119190, 891951]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [89]:
result_ = pd.read_csv('predictions_basic.csv',)
result_['actual'] = result['actual']
result_['random_recommendation'] = result_['random_recommendation'].str.strip('[]')
result_['popular_recommendation'] = result_['popular_recommendation'].str.strip('[]')
result_['itemitem'] = result_['itemitem'].str.strip('[]')
result_['cosine'] = result_['cosine'].str.strip('[]')
result_['tfidf'] = result_['tfidf'].str.strip('[]')
result_['own_purchases'] = result_['own_purchases'].str.strip('[]')
result_.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","5586238, 1015228, 866118, 2416733, 2603573","6534178, 6533889, 1029743, 6534166, 1082185","981760, 1127831, 1098066, 826249, 878996","981760, 1127831, 1098066, 878996, 826249","981760, 1127831, 1098066, 826249, 878996","999999, 1082185, 1029743, 995785, 1004906"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","161354, 63027, 1027802, 12263694, 307395","6534178, 6533889, 1029743, 6534166, 1082185","981760, 995242, 1029743, 840361, 961554","981760, 1004906, 961554, 1096036, 1080414","981760, 1004906, 859075, 1096036, 961554","999999, 1082185, 1098066, 6534178, 1127831"


In [59]:
# your_code

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [91]:
precision_rnd = result_.apply(lambda row: precision_at_k(list(map(int, row['random_recommendation'].split(sep=', '))), row['actual']), axis=1).mean()
precision_pop = result_.apply(lambda row: precision_at_k(list(map(int, row['popular_recommendation'].split(sep=', '))), row['actual']), axis=1).mean()
precision_item = result_.apply(lambda row: precision_at_k(list(map(int, row['itemitem'].split(sep=', '))), row['actual']), axis=1).mean()
precision_cos = result_.apply(lambda row: precision_at_k(list(map(int, row['cosine'].split(sep=', '))), row['actual']), axis=1).mean()
precision_idf = result_.apply(lambda row: precision_at_k(list(map(int, row['tfidf'].split(sep=', '))), row['actual']), axis=1).mean()
precision_own = result_.apply(lambda row: precision_at_k(list(map(int, row['own_purchases'].split(sep=', '))), row['actual']), axis=1).mean()

print('''
Precision@5:

random_recommendation = {0},
popular_recommendation = {1},
itemitem = {2},
cosine = {3},
tfidf = {4},
own_purchases = {5}
'''.format(precision_rnd, precision_pop, precision_item, 
           precision_cos, precision_idf, precision_own))


Precision@5:

random_recommendation = 0.0005876591576885406,
popular_recommendation = 0.15523996082272082,
itemitem = 0.033594515181195064,
cosine = 0.03525954946131262,
tfidf = 0.03614103819784544,
own_purchases = 0.1799869409076044



Лучшим алгоритмом является "Свои покупки" с Precision@5 = 0.1799, за ним идет "Популярные" с Precision@5 = 0.155, далее все три детерминированных алгоритма по убывающей "itemitem", "cosine", "tfidf"  с Precision@5 = 0.036, 0.035, 0.033

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.